In [1]:
# Iniciar sesion spark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("ETL").getOrCreate()
sc = spark.sparkContext

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
0,application_1710371625044_0001,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
# Carga de dataset desde S3
s3_path = "s3://mramosotrabajo1/dataCVS/Defunciones/Defunciones_Calculadas.csv"

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
# Lectura 
df = spark.read.csv(s3_path, header=True, sep= ',') # Alternar entre ',' y ';' según las características del csv

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
# Descripción de los datos en crudo
df.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

713

In [5]:
df.columns

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['Municipio', 'Periodo', 'Sexo', 'Edad', 'Capitulo', 'Grupo', 'Muertes']

In [7]:
df.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+-------+------+-------------+--------------------+--------------------+-------+
|Municipio|Periodo|  Sexo|         Edad|            Capitulo|               Grupo|Muertes|
+---------+-------+------+-------------+--------------------+--------------------+-------+
|  Barbosa|   2013| Mujer|Mayores de 65|Enfermedad circul...|Enfermedades isqu...|   7,32|
|  Barbosa|   2014|Hombre|Mayores de 65|Enfermedad circul...|Enfermedades isqu...|  17,74|
|  Barbosa|   2013|Hombre|Mayores de 65|Enfermedad circul...|Enfermedades isqu...|  10,44|
|  Barbosa|   2010|Hombre|Mayores de 65|Enfermedad respir...|Influenza [gripe]...|   2,08|
|  Barbosa|   2012|Hombre|Mayores de 65|Enfermedad respir...|Influenza [gripe]...|   4,14|
+---------+-------+------+-------------+--------------------+--------------------+-------+
only showing top 5 rows

In [15]:
df.describe().show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+---------+------------------+------+-------------+--------------------+--------------------+------------------+
|summary|Municipio|           Periodo|  Sexo|         Edad|            Capitulo|               Grupo|           Muertes|
+-------+---------+------------------+------+-------------+--------------------+--------------------+------------------+
|  count|      713|               713|   713|          713|                 713|                 713|               713|
|   mean|     null|2012.4936886395512|  null|         null|                null|                null|1.7692307692307692|
| stddev|     null|2.8864808865704377|  null|         null|                null|                null| 1.031802008437367|
|    min|  Barbosa|              2008|Hombre|Mayores de 65|Enfermedad circul...|Enfermedades cere...|                 1|
|    max| Sabaneta|              2017| Mujer|   Menor de 5|Enfermedad respir...|Otras infecciones...|             99,04|
+-------+---------+-------------

In [8]:
df.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- Municipio: string (nullable = true)
 |-- Periodo: string (nullable = true)
 |-- Sexo: string (nullable = true)
 |-- Edad: string (nullable = true)
 |-- Capitulo: string (nullable = true)
 |-- Grupo: string (nullable = true)
 |-- Muertes: string (nullable = true)

In [9]:
# Inicio de actividades ETL
from pyspark.sql.types import StringType,DoubleType,IntegerType,FloatType
df = df.withColumn("Muertes", df["Muertes"].cast(FloatType())) # Cambio de tipo de datos

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
df.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- Municipio: string (nullable = true)
 |-- Periodo: string (nullable = true)
 |-- Sexo: string (nullable = true)
 |-- Edad: string (nullable = true)
 |-- Capitulo: string (nullable = true)
 |-- Grupo: string (nullable = true)
 |-- Muertes: float (nullable = true)

In [11]:
df = df.filter(df['Muertes'].isNotNull()) # Eliminación de valores vacíos

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [25]:
df.describe().show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+---------+------------------+------+----------+--------------------+--------------------+------------------+
|summary|Municipio|           Periodo|  Sexo|      Edad|            Capitulo|               Grupo|           Muertes|
+-------+---------+------------------+------+----------+--------------------+--------------------+------------------+
|  count|       26|                26|    26|        26|                  26|                  26|                26|
|   mean|     null|2012.8846153846155|  null|      null|                null|                null|1.7692307692307692|
| stddev|     null| 3.089685072325949|  null|      null|                null|                null| 1.031802008437367|
|    min|   Caldas|              2008|Hombre|Menor de 5|Enfermedad circul...|Enfermedades cere...|               1.0|
|    max| Medellin|              2017| Mujer|Menor de 5|Enfermedad respir...|Otras infecciones...|               4.0|
+-------+---------+------------------+------+----------+

In [13]:
etl_muertes= df.groupBy('Municipio').agg({'Muertes':'sum'}) # Suma de muertes por municipio 

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
etl_muertes.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+------------+
|  Municipio|sum(Muertes)|
+-----------+------------+
|La Estrella|         1.0|
| Copacabana|         2.0|
|     Caldas|         1.0|
|   Medellin|        35.0|
|     Itagui|         7.0|
+-----------+------------+

In [26]:
etl_sexo= df.groupBy('Sexo').agg({'Muertes':'count'}) # Cuenta de muertes por sexo 
etl_sexo.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+--------------+
|  Sexo|count(Muertes)|
+------+--------------+
|Hombre|            12|
| Mujer|            14|
+------+--------------+

In [28]:
etl_Capitulo= df.groupBy('Capitulo').agg({'Muertes':'count'}) # Cuenta de muertes por edad 
etl_Capitulo.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------+
|            Capitulo|count(Muertes)|
+--------------------+--------------+
|Enfermedad respir...|            25|
|Enfermedad circul...|             1|
+--------------------+--------------+

In [33]:
# Filtro por capítulo y municipio para ver el sexo, edad y muertes
df.filter(df['Capitulo']=='Enfermedad respiratoria').filter(df['Municipio'] == 'Medellin').select('Sexo','Edad','Muertes').show(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+----------+-------+
|  Sexo|      Edad|Muertes|
+------+----------+-------+
|Hombre|Menor de 5|    1.0|
| Mujer|Menor de 5|    1.0|
| Mujer|Menor de 5|    1.0|
| Mujer|Menor de 5|    2.0|
|Hombre|Menor de 5|    4.0|
| Mujer|Menor de 5|    3.0|
| Mujer|Menor de 5|    1.0|
| Mujer|Menor de 5|    3.0|
| Mujer|Menor de 5|    1.0|
| Mujer|Menor de 5|    1.0|
+------+----------+-------+
only showing top 10 rows

In [14]:
output_path = "s3://mramosotrabajo1/results/"

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
etl_muertes.write.csv(output_path + 'salida_etl', header=True, mode="overwrite")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…